<h1><center>CMPE 462 - Project 1<br>Binary Classification with Logistic Regression<br>Due: April 23, 2020, 23:59</center></h1>

* **Student ID1: 2016400198**
* **Student ID2: 2016400126**
* **Student ID3: 2018400279**

# Overview

In this project, you are going to implement logistic regression from scratch. You are provided
a subset of the famous handwritten digit dataset called MNIST. In the subset, you will find images of digit 1 and 5. Therefore, you will be solving a binary classification problem. The project includes feature extraction, model training, and evaluation steps.

First, you will load and visualize the data portion we have provided to you and then extract two different set of features to build a classifier on. When you extracted the desired features, you will run your logistic regression implementation with gradient descent on the representations to classify digits into 1 and 5. You will experiment with different learning rates  and regularization parameter ($\lambda$) and find the optimal $\lambda$ with 5-fold cross validation. Finally, you will evaluate the implemented models, decide which is the best performing one and visualize a decision boundary.

Follow the steps on this notebook that would guide you through the solution step-by-step. Make sure that the notebook reports your work properly and add comments and opinions when necessary.

**IMPORTANT NOTE:** You are allowed to use third-party libraries such as `numpy` and `matplotlib` to help you implement necessary procedures. However, you should not import any function that accomplishes the task itself. For instance, you can use `numpy` arrays for matrix operations, but you cannot use `scikit-learn` to implement cross validation.

## Task 1: Feature Extraction (35 Pts)

Load the training/test data and labels as numpy arrays (Hint:`np.load`). Train and test data are 1561x256 and 424x256 dimensional matrices, respectively. Each row in the
aforementioned matrices corresponds to an image of a digit. The 256 pixels correspond to a 16x16 image. Label 1 is assigned to digit 1 and label -1 is assigned to digit 5.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X_train = np.load("data/train_data.npy")
X_test = np.load("data/test_data.npy")
y_train = np.load("data/train_labels.npy")
y_test = np.load("data/test_labels.npy")

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

**(5 points)** Display two of the digit images, one for digit 1 and one for digit 5. You can use `imshow` function of `matplotlib` library with a suitable colormap. You will first need to reshape 256 pixels to a 16x16 matrix. 

In [ ]:
fig = plt.figure(figsize=(5,5))
fig.add_subplot(1,2,1)
plt.imshow(np.reshape(X_train[0],(16,16)),cmap="gray")
fig.add_subplot(1,2,2)
plt.imshow(np.reshape(X_train[-1],(16,16)),cmap="gray")
plt.show()

**(10 points) Implementing Representation 1:** Extract the **symmetry** and **average intensity** features discussed in the class (see logistic regression lecture notes). To compute the intensity features, compute the average pixel value of the image, and for the symmetry feature, compute the negative of the norm of the difference between the image and its y-axis symmetrical. Search numpy's documentation to find the suitable function at each step. You should extract these two features for each image in the training and test sets. As a result, you should obtain a training data matrix of size 1561x2 and test data matrix of size 424x2.

Throughout the notebook, we will refer the representation with these two features as **Representation 1** 

In [ ]:
rep_1_train = np.zeros((len(X_train),2)) # first index -> symmetry, second index -> intensity
rep_1_test = np.zeros((len(X_test),2)) # first index -> symmetry, second index -> intensity

# Intensity
rep_1_train[:,1] = np.sum(X_train,axis=1)
rep_1_test[:,1] = np.sum(X_test,axis=1)

# Symmetry
for i in range(len(X_train)):
    x_train = np.reshape(X_train[i],(16,16))
    x_train_flipped = np.flip(x_train,axis=1) 
    rep_1_train[i,0] = np.sum(-1*abs(x_train-x_train_flipped))

for i in range(len(X_test)):
    x_test = np.reshape(X_test[i],(16,16))
    x_test_flipped = np.flip(x_test,axis=1)
    rep_1_test[i,0] = np.sum(-1*abs(x_test-x_test_flipped))


**(5 points)** Provide two scatter plots, one for training and one for test data. The plots should contain the average intensity values in the x-axis and symmetry values in the
y-axis. Denote the data points of label 1 with blue marker shaped <font color='blue'>o</font> and the data points of label -1 with a red marker shaped <font color='red'>x</font>. (Hint: check out `plt.scatter` and its `marker` and `color` parameters). Explicitly state the axis labels and figure title for both plots (Hint: `plt.xlabel`, `plt.ylabel`, `plt.title`). 

In [ ]:
fig = plt.figure(figsize=(20,10))

fig.add_subplot(1,2,1)
plt.scatter(rep_1_train[y_train == 1][:,0],rep_1_train[y_train == 1][:,1],c="r")
plt.scatter(rep_1_train[y_train == -1][:,0],rep_1_train[y_train == -1][:,1],c="b")
plt.xlabel("symmetry value")
plt.ylabel("intensity value")
plt.title("Train Data")

fig.add_subplot(1,2,2)
plt.scatter(rep_1_test[y_test == 1][:,0],rep_1_test[y_test == 1][:,1],c="r")
plt.scatter(rep_1_test[y_test == -1][:,0],rep_1_test[y_test == -1][:,1],c="b")
plt.xlabel("symmetry value")
plt.ylabel("intensity value")
plt.title("Test Data")

plt.show()

### There is a outlier at the bottom-right of the training data plot. We want to visualize that outlier.

In [ ]:
class_0 = rep_1_train[y_train==-1]  # Take all data of first class
outlier_index = np.argmax(class_0[:,0]>-50)  # Find the index of outlier

for a,i in enumerate(rep_1_train[:,0]==class_0[outlier_index][0]):
    if i:
        plt.imshow(np.reshape(X_train[1517],(16,16)),cmap="gray")
        break

### This data was labeled as 5, however it doesn't seem like 5 or 1.

**(15 points) Implementing Representation 2:** Come up with an alternative feature extraction approach. The features can again be 2-D, or higher dimensional. If you use any external resource, please cite the references. Explain the feature extraction procedure clearly in your report; if it is an algorithm, provide the algorithm; if it is a function, provide the mathematical expressions. 

If your proposed features are 2-D or 3-D, provide the scatter plots similar to the previous step.

We will refer this representation proposed by you as **Representation 2**.

#### We chose 2 features. 
#### First one is the variance of the x coordinates of white pixels. We expect that this feature will be low for "1" images and high for "5" images.
#### Second one is the distance on x axis between the white pixel with maximum x coordinate value and the one with minimum x coordinate value.

In [ ]:
rep_2_train = np.zeros((len(X_train),2))
rep_2_test = np.zeros((len(X_test),2))

          
for index, x in enumerate(X_train.reshape(-1,16,16)):
    x_indices = []
    indices = x > 0
    
    # Find white pixels and store their x coordinates.
    for i in range(len(indices)):
        for j in range(len(indices[i])):
            if indices[i][j]:  
                x_indices.append(j)

    rep_2_train[index,0] = np.var(x_indices)
    rep_2_train[index,1] = max(x_indices) - min(x_indices)


for index, x in enumerate(X_test.reshape(-1,16,16)):
    x_indices = []
    indices = x > 0

    # Find white pixels and store their x coordinates.
    for i in range(len(indices)):
        for j in range(len(indices[i])):
            if indices[i][j]:
                x_indices.append(j)

    rep_2_test[index,0] = np.var(x_indices)
    rep_2_test[index,1] = max(x_indices) - min(x_indices)


fig = plt.figure(figsize=(20,10))

fig.add_subplot(1,2,1)
plt.scatter(rep_2_train[y_train == 1][:,0],rep_2_train[y_train == 1][:,1],c="r")
plt.scatter(rep_2_train[y_train == -1][:,0],rep_2_train[y_train == -1][:,1],c="b")
plt.xlabel("variance value")
plt.ylabel("difference of x coordinates")
plt.title("Train Data")

fig.add_subplot(1,2,2)
plt.scatter(rep_2_test[y_test == 1][:,0],rep_2_test[y_test == 1][:,1],c="r")
plt.scatter(rep_2_test[y_test == -1][:,0],rep_2_test[y_test == -1][:,1],c="b")
plt.xlabel("variance value")
plt.ylabel("difference of x coordinates")
plt.title("Test Data")

plt.show()


## Task 2: Logistic Regression (40 Pts)

**(20 points)** Implement the logistic regression classifier from scratch with gradient descent and train it using Representation 1 and Representation 2 as inputs. Concatenate
1 to your features for the intercept term, such that one data point will look like for 2-D features [1,$x_1$,$x_2$], and the model vector will be [$w_0, w_1, w_2$], where $w_0$ is the intercept parameter. 
You can refer to lecture notes (Logistic regression slides 29-30) to review the gradient descent learning algorithm and the logistic loss. To implement the gradient of the logistic loss with respect to $w$, first derive its expression by hand. Please include your derivation in your report.

To prove that your implementation is converging, keep the loss values at each gradient descent iteration in a numpy array. After the training is finalized, plot the loss values
with respect to iteration count (Hint: `plt.plot`). You should observe a decreasing loss as the number of iterations increases. Also, experiment with 5 different learning rates between 0 and 1, and plot the convergence curves for each learning rate in the same plot to observe the effect of the learning rate (step size) on the convergence. 

To decide when to terminate the gradient descent iterations, check the absolute difference between the current loss value and the loss value of the previous step. If the difference is less than a small number, such as $10^{-5}$, you can exit the loop.

### Data Preparation

In [ ]:
X_train_1 = np.concatenate((np.ones((len(rep_1_train),1)), rep_1_train), axis=1)
X_test_1 = np.concatenate((np.ones((len(rep_1_test),1)), rep_1_test), axis=1)

X_train_2 = np.concatenate((np.ones((len(rep_2_train),1)), rep_2_train), axis=1)
X_test_2 = np.concatenate((np.ones((len(rep_2_test),1)), rep_2_test), axis=1)

print("Data Summary")
print("1st Representation -> X-train = {}, y-train = {}, X-test = {}, y-test = {}".format(str(X_train_1.shape), str(y_train.shape), str(X_test_1.shape), str(y_test.shape)))
print("2nd Representation -> X-train = {}, y-train = {}, X-test = {}, y-test = {}".format(str(X_train_2.shape), str(y_train.shape), str(X_test_2.shape), str(y_test.shape)))

### Useful Functions

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))


def logistic_loss(y_pred, y_true, reg_lambda=None, w=None):
    if reg_lambda:
        return np.sum(np.log(1 + np.exp(np.multiply(y_pred, -y_true))))/len(y_true) + reg_lambda*np.sum(w.dot(w))
    else:
        return np.sum(np.log(1 + np.exp(np.multiply(y_pred, -y_true))))/len(y_true)


# Fits logistic regression model to given data. 
# Returns weight matrix and loss history.
def fit_logistic_model(X, y, epoch, learning_rate, reg_lambda=None):
    
    w = np.zeros((len(X[0])))
    loss_history = []

    for step in range(epoch):

        y_pred = X.dot(w.T)

        if reg_lambda:
            loss = logistic_loss(y_pred, y, reg_lambda, w)
        else:
            loss = logistic_loss(y_pred, y)

        grad_1 = (1 / (1 + np.exp(np.multiply(y, X.dot(w.T)))))
        grad_2 = (-1 * (np.reshape(y, (-1,1)) * X))
        grad = np.reshape(grad_1, (-1,1)) * grad_2
        grad = np.sum(grad,axis=0)/len(y)

        if reg_lambda:
            grad = grad + 2*reg_lambda*w

        w[:] = w - learning_rate*grad
        loss_history.append(loss)

        # Early Stopping check after 50th epoch
        if len(loss_history) > 50 and loss_history[-2]-loss_history[-1]<1e-5:
            break
    
    return [w, loss_history]

### Train on Rep 1

In [ ]:
fig = plt.figure(figsize=(25,5))

lr_list = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]

for i, lr in enumerate(lr_list):
    fig.add_subplot(1,5,i+1)
    w, losses = fit_logistic_model(X_train_1, y_train, epoch=5000, learning_rate=lr)
    plt.scatter(range(len(losses)), losses)
    plt.title("LR = "+str(lr)+" Loss = "+str(losses[-1])[:6])

plt.show()

### Train on Rep 2

In [ ]:
fig = plt.figure(figsize=(25,5))

# Larger lr worked better for this case
lr_list = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]

for i, lr in enumerate(lr_list):
    fig.add_subplot(1,5,i+1)
    w, losses = fit_logistic_model(X_train_2, y_train, epoch=5000, learning_rate=lr)
    plt.scatter(range(len(losses)), losses)
    plt.title("LR = "+str(lr)+" Loss = "+str(losses[-1])[:6])

plt.show()

**(10 points)** Implement logistic regression with $\ell_2$ norm regularization, $||\mathbf{w}||_{2}^{2}$ . Show that your implementation is working by visualizing the loss over the iterations again. Visualization for a single learning rate and $\lambda$ suffices. 

In [ ]:
# Data was set and train function was already written above.

fig = plt.figure(figsize=(10,5))

# Rep 1
fig.add_subplot(1,2,1)
lr = 1e-3
w, losses = fit_logistic_model(X_train_1, y_train, epoch=5000, learning_rate=lr, reg_lambda=0.001)
plt.scatter(range(len(losses)), losses)
plt.title("Rep 1 - LR = "+str(lr)+", Loss = "+str(losses[-1])[:6] + ", Reg = 0.001")

# Rep 2
fig.add_subplot(1,2,2)
lr = 1e-3
w, losses = fit_logistic_model(X_train_2, y_train, epoch=5000, learning_rate=lr, reg_lambda=0.001)
plt.scatter(range(len(losses)), losses)
plt.title("Rep 2 - LR = "+str(lr)+", Loss = "+str(losses[-1])[:6] + ", Reg = 0.001")

plt.show()

**(10 points)** Implement a 5-fold cross validation procedure to find the optimal $\lambda$ value for both Representation 1 and 2. Experiment with at least three different $\lambda$ values between 0 and 1. Report the mean/std of cross validation accuracy of every representation/parameter combination as a table and clearly mark the best configuration in your report. 

### Functions

In [ ]:
# Returns accuracy of predictions made by given weigths.
def accuracy(X, y, w):
    correct = np.sum(np.logical_not(np.logical_xor(sigmoid(X.dot(w))>0.5, y==1)))
    wrong = len(y) - correct
    return correct/len(y)


# Merges given lists except the one at "index"
def merger(input_list, index):
    result = []
    for i in range(len(input_list)):
        if i is not index:
            result.extend(input_list[i])
    return result


## CV for 1st Representation

### Split the train data for 5-fold cross validation

In [ ]:
splitted_data = []
splitted_truth = []

for i in range(5):
    splitted_data.append([])
    splitted_truth.append([])

# For randomness, we split the data with the following way using mod
for i in range(X_train_1.shape[0]):
    splitted_data[i%5].append(X_train_1[i])
    splitted_truth[i%5].append(y_train[i])

### Cross Validation Implementation

In [ ]:
means = []
stds = []
training_result_list = []
lambdas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]

for i in range(len(lambdas)):
    acc_hist = []
    for j in range(5):
        validation_list = splitted_data[j]
        validation_truth = splitted_truth[j]
        folded_rep_1_training_data = merger(splitted_data,j)
        folded_rep_1_truth = merger(splitted_truth,j)
        training_result = fit_logistic_model(np.array(folded_rep_1_training_data), np.array(folded_rep_1_truth), epoch=5000, learning_rate=1e-3, reg_lambda=lambdas[i])
        acc_hist.append(accuracy(np.array(validation_list), np.array(validation_truth), training_result[0]))
        training_result_list.append(training_result)
    
    means.append(np.mean(acc_hist))
    stds.append(np.std(acc_hist))

print("LR = 1e-3")
for i in range(len(lambdas)):
    print("Rep. 1 -> Lambda = {}, Avg Accuracy = {}, Std of Accuracy = {}".format(lambdas[i], means[i], stds[i]))

print("################################################################################################")
best_lambda_i = np.argmax(means)
print("Rep. 1 -> Best Lambda = {}, Best Accuracy = {}".format(lambdas[best_lambda_i], means[best_lambda_i]))


## CV for 2nd Representation

### Split the train data for 5-fold cross validation

In [ ]:
splitted_data = []
splitted_truth = []

for i in range(5):
    splitted_data.append([])
    splitted_truth.append([])

# For randomness, we split the data with the following way using mod
for i in range(X_train_2.shape[0]):
    splitted_data[i%5].append(X_train_2[i])
    splitted_truth[i%5].append(y_train[i])

### Cross Validation Implementation

In [ ]:
means = []
stds = []
training_result_list = []
lambdas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]

for i in range(len(lambdas)):
    acc_hist = []
    for j in range(5):
        validation_list = splitted_data[j]
        validation_truth = splitted_truth[j]
        folded_rep_2_training_data = merger(splitted_data,j)
        folded_rep_2_truth = merger(splitted_truth,j)
        training_result = fit_logistic_model(np.array(folded_rep_2_training_data), np.array(folded_rep_2_truth), epoch=5000, learning_rate=1e-3, reg_lambda=lambdas[i])
        acc_hist.append(accuracy(np.array(validation_list), np.array(validation_truth), training_result[0]))
        training_result_list.append(training_result)
    
    means.append(np.mean(acc_hist))
    stds.append(np.std(acc_hist))

print("LR = 1e-3")
for i in range(len(lambdas)):
    print("Rep. 2 -> Lambda = {}, Avg Accuracy = {}, Std of Accuracy = {}".format(lambdas[i], means[i], stds[i]))

print("################################################################################################")
best_lambda_i = np.argmax(means)
print("Rep. 2 -> Best Lambda = {}, Best Accuracy = {}".format(lambdas[best_lambda_i], means[best_lambda_i]))


## Task 3: Evaluation (25 Pts)

**(5 points)** Train the logistic regression classifier on Representation 1 and 2 with the best learning rate you decide. Similarly, train the regularized logistic regression classifier with the best $\lambda$ you obtained by 5-fold cross validation. Report the training and test classification accuracy as 
\begin{align*}
\frac{\text{number of correctly classified samples}}{\text{total number of samples}}x100
\end{align*}

### Best parameters for rep. 1 and Rep. 2:
### LR = 1e-3, Reg = 0.1

In [ ]:
w_1_no_reg, _ = fit_logistic_model(X_train_1, y_train, 5000, learning_rate=1e-3, reg_lambda=None)
w_2_no_reg, _ = fit_logistic_model(X_train_2, y_train, 5000, learning_rate=1e-3, reg_lambda=None)

w_1_reg, _ = fit_logistic_model(X_train_1, y_train, 5000, learning_rate=1e-3, reg_lambda=0.1)
w_2_reg, _ = fit_logistic_model(X_train_2, y_train, 5000, learning_rate=1e-3, reg_lambda=0.1)

print("TRAIN")
print("Rep. 1 Accuracy: ", accuracy(X_train_1, y_train, w_1_no_reg)*100)
print("Rep. 1 Accuracy (Reg): ", accuracy(X_train_1, y_train, w_1_reg)*100)
print("Rep. 2 Accuracy: ", accuracy(X_train_2, y_train, w_2_no_reg)*100)
print("Rep. 2 Accuracy (Reg): ", accuracy(X_train_2, y_train, w_2_reg)*100)

print("\nTEST")
print("Rep. 1 Accuracy: ", accuracy(X_test_1, y_test, w_1_no_reg)*100)
print("Rep. 1 Accuracy (Reg): ", accuracy(X_test_1, y_test, w_1_reg)*100)
print("Rep. 2 Accuracy: ", accuracy(X_test_2, y_test, w_2_no_reg)*100)
print("Rep. 2 Accuracy (Reg): ", accuracy(X_test_2, y_test, w_2_reg)*100)

**(15 points)** Visualize the decision boundary (the line that is given by $\mathbf{w}^{T}x=0$) obtained from the logistic regression classifier learned without regularization. For this purpose, use only Representation 1. Provide two scatter plots for training and test data points with the decision boundary shown on each of the plots.

In [ ]:
x = np.linspace(-250,0,10)
y = x*(-w_1_no_reg[1]/w_1_no_reg[2]) + (-w_1_no_reg[0]/w_1_no_reg[2])

fig = plt.figure(figsize=(20,10))

fig.add_subplot(1,2,1)
plt.scatter(rep_1_train[y_train == 1][:,0],rep_1_train[y_train == 1][:,1])
plt.scatter(rep_1_train[y_train == -1][:,0],rep_1_train[y_train == -1][:,1])
plt.plot(x, y, "r--")
plt.xlabel("symmetry value")
plt.ylabel("intensity value")
plt.title("Train Data")

fig.add_subplot(1,2,2)
plt.scatter(rep_1_test[y_test == 1][:,0],rep_1_test[y_test == 1][:,1])
plt.scatter(rep_1_test[y_test == -1][:,0],rep_1_test[y_test == -1][:,1])
plt.plot(x, y, "r--")
plt.xlabel("symmetry value")
plt.ylabel("intensity value")
plt.title("Test Data")

plt.show()

**(5 points)** Comment on your work in your report. Include the answers for the following questions in your discussion. 

* Did regularization improve the generalization performance (did it help reducing the gap between training and test accuracies/errors)? Did you observe any difference between using Representation 1 and 2?
* Which feature set did give the best results? Which one is more discriminative?
* What would be your next step to improve test accuracy? 